<a href="https://colab.research.google.com/github/taravatp/Deep_Learning_Practices/blob/main/training/train_semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing all the required packages and libraries

In [ ]:
!pip install import_ipynb 

In [ ]:
cd /content/drive/MyDrive/panoptic_FPN

In [ ]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

import import_ipynb
import time

import matplotlib.pyplot as plt
import numpy as np
from coco_semantic_dataset import coco_semantic_dataset
from models.semantic_model import SemanticSegmentationModel

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:',device)

device: cuda


#Instantiating the Dataset/Model/Optimizer/HyperParameters

In [ ]:
batch_size = 8
dataset_train = coco_semantic_dataset(flag='train')
dataset_test = coco_semantic_dataset(flag='test')

data_loader_train = torch.utils.data.DataLoader(dataset_train,batch_size=batch_size,shuffle=True,num_workers=2)
data_loader_test = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size,shuffle=True,num_workers=2) 

In [ ]:
model = SemanticSegmentationModel(len(dataset_train.category_IDS)).to(device)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [ ]:
learning_rate = 1e-4
num_epochs =30
#lr 1e-4 az hamoon epoche aval be amalkarde behtari resid ta 1e-3

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=learning_rate, weight_decay=0.0002)
loss_function = nn.CrossEntropyLoss()

In [ ]:
print(len(dataset_train))

1500


# Training

In [ ]:
number_batches = len(data_loader_train)
loss_per_epoch = []
for epoch in range(num_epochs):

  print('starting epoch number:',epoch)
  time_start = time.time()
  model.train()
  loss_segmentation = 0

  for iter,batch in enumerate(data_loader_train):

    images = batch[0].to(device)
    segmentation_masks = batch[1].to(device).long()

    predicted_masks = model(images) #[N,NUM_CLASSES,H,W]    

    loss = loss_function(predicted_masks,segmentation_masks)
    loss_segmentation += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if iter%100 == 0 :
      print(f"epoch:{epoch} *** iter:{iter} *** batach:{iter} *** segmentation loss:{loss}")
  
  #lr_scheduler.step()

  loss_per_epoch.append(loss_segmentation/len(data_loader_train))
  print(f"end of epoch number {epoch}")
  print(f"time elapsed: {time.time() - time_start}")
  print(f"train_loss: {loss_segmentation/len(data_loader_train)}")
  torch.save(model.state_dict(), f"semantic_model{epoch}_batch_{batch_size}.pth")



starting epoch number: 0
epoch:0 *** iter:0 *** batach:0 *** segmentation loss:4.764401912689209
epoch:0 *** iter:100 *** batach:100 *** segmentation loss:2.6676666736602783
end of epoch number 0
time elapsed: 329.51360607147217
train_loss: 2.495858284387183
starting epoch number: 1
epoch:1 *** iter:0 *** batach:0 *** segmentation loss:2.2049081325531006
epoch:1 *** iter:100 *** batach:100 *** segmentation loss:2.625758647918701
end of epoch number 1
time elapsed: 304.2438576221466
train_loss: 1.9806828961727467
starting epoch number: 2
epoch:2 *** iter:0 *** batach:0 *** segmentation loss:1.7764049768447876
epoch:2 *** iter:100 *** batach:100 *** segmentation loss:1.7360669374465942
end of epoch number 2
time elapsed: 303.93553948402405
train_loss: 1.7424557535572256
starting epoch number: 3
epoch:3 *** iter:0 *** batach:0 *** segmentation loss:1.6466827392578125
epoch:3 *** iter:100 *** batach:100 *** segmentation loss:1.5396018028259277
end of epoch number 3
time elapsed: 303.980798

In [ ]:
plt.plot(loss_per_epoch)

# to do list


1.   ye bare dg ba tedad epoche bishtar amoozesh bede -> 40 is excellent
2.   learning rate ro yekam bishtar kon ( masalan zarbdar 2) shayad zood tar be natije beresi -> overshoots the minimum
3.   ba ziad kardane batch size natijat behtar shod! batch size ro bozorgtar ham kon(16) bebin tasiri dare ya na! --> cuda out of memory:|

